# 1 - Introduction

This notebook train four different models that make use of the pretrained embeddings with 50 dimensions.

## 1.1 Load Packages and Global Variables

In [1]:
%matplotlib inline
import os
import numpy as np
import nltk
from sklearn.externals import joblib
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

Using TensorFlow backend.


In [2]:
PROJECT_DIR = os.path.join(os.getcwd(), os.pardir)
os.chdir(PROJECT_DIR)

In [3]:
import src.neural_networks as nn

In [4]:
from importlib import reload
nn = reload(nn)

## 1.2- Load the Data

Load the train and validation neural network data sets if they are present, otherwise raise an exception.

In [5]:
#Load sequnces
try:
    train = joblib.load('data/processed/train_nn.pkl')
    valid = joblib.load('data/processed/valid_nn.pkl')
except FileNotFoundError:
    #need to run earlier notebook if files not present
    raise Exception("Files not found. Run Notebook 4.1")

In [6]:
#Load embedding matrix of 50 dimensions
try:
    embedding_matrix50 = joblib.load('data/interim/embeddings50.pkl')
except FileNotFoundError:
    #need to run earlier notebook if files not present
    raise Exception("Files not found. Run Notebook 4.1")

In [7]:
#Load embedding matrix of 300 dimensions
try:
    embedding_matrix300 = joblib.load('data/interim/embeddings300.pkl')
except FileNotFoundError:
    #need to run earlier notebook if files not present
    raise Exception("Files not found. Run Notebook 4.1")

In [8]:
#these arguments will be the same when training both models
run_args = {"train":(train["seqs"],train["labels"]),
            "valid":(valid["seqs"],valid["labels"])}

# 2 - Train 50d model with fixed embeddings

In [9]:
#LSTM, average final
model50_fixed = nn.create_model(embedding_matrix=embedding_matrix50, n_hidden=150, train_embed=False)
model50_fixed.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
joke_seq (InputLayer)        (None, 300)               0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 50)           1369800   
_________________________________________________________________
mask_paddings (Masking)      (None, 300, 50)           0         
_________________________________________________________________
drop_words (SpatialDropout1D (None, 300, 50)           0         
_________________________________________________________________
mask_dropped_words (Masking) (None, 300, 50)           0         
_________________________________________________________________
reccurrent_layer (LSTM)      (None, 150)               120600    
_________________________________________________________________
drop_dense (Dropout)         (None, 150)               0         
__________

In [10]:
#output model as svg plot
plot_model(model50_fixed, to_file='reports/figures/nn_50d.svg', show_shapes =True)

In [ ]:
%%time
#now train
history50_fixed = nn.run_model(model=model50_fixed, out_path="models/nn_50d_fixed.hdf5", **run_args)

Train on 171945 samples, validate on 24564 samples
Epoch 1/1000
171945/171945 [==============================] - 135s 788us/step - loss: 0.6722 - acc: 0.5764 - val_loss: 0.6591 - val_acc: 0.6049
Epoch 2/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.6627 - acc: 0.5952 - val_loss: 0.6532 - val_acc: 0.6050
Epoch 3/1000
171945/171945 [==============================] - 90s 524us/step - loss: 0.6600 - acc: 0.5973 - val_loss: 0.6533 - val_acc: 0.6059
Epoch 4/1000
171945/171945 [==============================] - 90s 525us/step - loss: 0.6559 - acc: 0.6024 - val_loss: 0.6525 - val_acc: 0.6110
Epoch 5/1000
168000/171945 [============================>.] - ETA: 1s - loss: 0.6547 - acc: 0.6022

KeyboardInterrupt: 

# 3 - Train 300d model with fixed embeddings

Second model 300 dimensionhal word embeddings. However the number of hidden units is decreases in order mtianta  aprocaml the same number of trainable paramters.

In [ ]:
#LSTM, average final
model300_fixed = nn.create_model(embedding_matrix=embedding_matrix300, n_hidden=150)
model300_fixed.summary()

In [ ]:
%%time
#now train
history300_fixed = nn.run_model(model=model300_fixed, out_path="models/nn_300d_fixed.hdf5", **run_args)

# 4 - Train 50d model with trainable embeds

In [ ]:
#LSTM, average final
model50_trained = nn.create_model(embedding_matrix=embedding_matrix50, n_hidden=150, train_embed=True)
model50_trained.summary()

In [ ]:
%%time
history50_trained = nn.run_model(model=model50_trained, out_path="models/nn_50d_trained.hdf5",**run_args)

# 5 - Train 300d model with trainable embeds

In [ ]:
#LSTM, average final
model300_trained = nn.create_model(embedding_matrix=embedding_matrix300, n_hidden=150, train_embed=True)
model300_trained.summary()

In [ ]:
%%time
history300_trained = nn.run_model(model=model300_trained, out_path="models/nn_300d_trained.hdf5", **run_args)

# 6 -  Analyze Best